In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import import_ipynb #requires pip install

Import new dataset

In [34]:
users = pd.read_csv(r"D:\dataset\encoding\user_frame.csv")
anime = pd.read_csv(r"D:\dataset\encoding\anime_frame.csv")
ratings = pd.read_csv(r"D:\dataset\encoding\ratings_frame.csv")
#Function to turn "genres" columns from string into list
def string_to_list(genres):
    return eval(genres)

users["fav_genres"] = users["fav_genres"].apply(string_to_list)
anime["genre"] = anime["genre"].apply(string_to_list)
ratings["genre"] = ratings["genre"].apply(string_to_list)

In [4]:
#Build genre list
def split(genres):
    return str(genres).split(",")

def build_genre_list(dataframe):
    #Updating the set to get the individual genre names in the set instead of the original list of genres
    genre_set = set()
    for i in dataframe: 
        genre_set.update(i)
    result = list(genre_set)

    #Removing blank spaces
    final_result = []
    for i in result:
        if i!="nan":
            new = i.replace(" ", "")
            final_result.append(new)

    #Removing duplicates now that spaces have been removed
    genre_list = list(set(final_result))
    return genre_list

In [5]:


user_cols = users.loc[:, ["username", "experience","gender", "generation",  "fav_anime_period"]]

encoder = LabelEncoder()
#Function to create a separate feature for each column and vectorize
def genre_columns():

    user_genres = users.loc[:, ["username", "fav_genres"]]
    genre_list = build_genre_list(user_genres["fav_genres"])
 
    for i in genre_list:
        user_cols[i] = 0

    for x, y in user_genres.iterrows():
        user_row = user_cols.loc[user_cols["username"] == y["username"]]
        #print(y)
        #encode each of the three favourite genres with value: 1
        user_row[y["fav_genres"][0]] = 1
        user_row[y["fav_genres"][1]] = 1
        user_row[y["fav_genres"][2]] = 1
        user_cols.loc[user_cols["username"] == y["username"]] = user_row
    #for genres in user_genres:
        #user_cols[genres[0]] = 1
        #ser_cols[genres[2]] = 1
        #print(genres)
        #bag_vector = tfidf.transform(genres)
        
        #print(bag_vector.toarray())
    genre_cols = user_cols.drop(columns=["experience","gender", "generation","fav_anime_period"])
    return genre_cols

#Vectorise features of a single user
def vectorize_single(user):
    #Get row of one user
    user_row = user_cols.loc[user_cols["username"] == user]
    user_vector = user_row.copy()
    
    #Fit the encoder with the data of original df: user_cols
    for col in user_cols.columns:
        enc = encoder.fit(user_cols[col])
        #print(list(enc.classes_))
        user_vector[col]=enc.transform(user_vector[col]) #encode each column of target user

    return user_vector

#Vectorise features of all users in the dataframe
def vectorize_all():
    all_users = user_cols.copy()

    for user in user_cols["username"]:
        user_vector = vectorize_single(user)
        all_users.loc[all_users["username"] == user] = user_vector
    #Make sure username field is in string form for identifier    
    all_users["username"] = user_cols["username"]
    #K-Nearest Neighbours
    #nb = KNeighborsClassifier(n_neighbors=5)
    return all_users

#Construct final vector of encoded features
def construct_vectors():
    string_features = vectorize_all()
    genre_features = genre_columns()
    final_vector = pd.merge(string_features, genre_features, on = "username")
    return final_vector

#construct_vectors().to_csv(r"D:\dataset\encoding\user_vectors.csv", index=False)
#genre_columns()
#vectorize_all()
#user_cols
# need to translate genres that user is asked to anime genres


In [6]:
vectorize_single("badking95")


,username,experience,gender,generation,fav_anime_period
9301,6443,3,1,1,1


In [7]:

anime_vector = anime.loc[:, ["airing","studio", "genre", "time_period", "fame"]]
anime_vector
#anime
#anime_genres = users.loc[:, "genre"]
#genre_list = build_genre_list(anime_genres)
#for i in genre_list:
            #user_cols[i] = 0

,airing,studio,genre,time_period,fame
0,False,David Production,"[Comedy, Supernatural, Romance, Shounen]",New-Gen,famous
1,False,Gonzo,"[Comedy, Parody, Romance, School, Shounen]",Classic,semi-famous
2,False,Satelight,"[Comedy, Magic, School, Shoujo]",Classic,well-known
3,False,Hal Film Maker,"[Comedy, Drama, Magic, Romance, Fantasy]",Classic,semi-famous
4,False,J.C.Staff,"[Comedy, Drama, Romance, Shounen]",New-Gen,semi-famous
...,...,...,...,...,...
3235,False,Millepensee,"[Music, Drama]",New-Gen,niche
3236,False,Production I.G,"[Comedy, Sports, School, Shounen]",New-Gen,famous
3237,False,"Satelight, Production Reed","[Action, Mecha, Romance, Super Power, Supe...",Classic,well-known
3238,False,Toei Animation,"[Adventure, Fantasy]",Classic,unknown


In [105]:
rf = pd.read_csv(r"D:\dataset\encoding\collab_frame.csv")
full_anime = pd.read_csv(r"D:\dataset\anime_cleaned.csv")

full_ratings = rf[rf["username"].isin(users["username"])] #ensure that only ratings from users that are in the users dataframe are included

print(len(full_ratings))

1626920


In [103]:


anime_matrix = pd.DataFrame(columns = users["username"].values)
anime_matrix["anime_id"] = anime["anime_id"]
anime_matrix.set_index("anime_id")
col = anime_matrix.pop("anime_id")
anime_matrix.insert(0, col.name, col)
 #list(users["username"].values)
#anime_matrix
#full_ratings.set_index("anime_id")
#anime_ratings = full_ratings.groupby("anime_id").head()
#anime_ratings = pd.DataFrame(anime_ratings)
#anime_ratings

#for show in anime_matrix.values:
    #show_id = int(full_anime.loc[full_anime["title"]==show[0]]["anime_id"])
    #anime_ratings = full_ratings.groupby("anime_id").get_group(show_id)
    #usernames = list(anime_ratings["username"])
    #scores = list(anime_ratings["my_score"]) #normalise ratings
    #print(show_row)
    #print(usernames)
    #print(scores)      

    #anime_matrix.loc[anime_matrix["anime"]==show[0],usernames] = scores
#anime_matrix
  
#for user in users["username"]:
    #anime_ratings = full_ratings.loc[full_ratings["username"]==user]


#user-item matrix with ratings normalised
for show in anime["anime_id"]:
    #print(show)
    for i, user in users.iterrows():
        username = user["username"]
        mean = float(user["stats_mean_score"])
        rating = full_ratings.loc[(full_ratings["username"]==username) & (anime["anime_id"]==show)]["my_score"]
        norm = rating - mean

        if rating.empty==False:
            anime_matrix.loc[anime_matrix["anime_id"]==show,username] = norm
        else:
            anime_matrix.loc[anime_matrix["anime_id"]==show,username] = 0

anime_matrix



KeyboardInterrupt: 

In [ ]:
a = list(set(anime_matrix.columns[1:]))
b = anime_matrix.columns[1:]
print(len(a), len(b))

c = pd.DataFrame(columns = users["username"].values)
c
#remove 0 ratings

108652 108652


,karthiga,Damonashu,bskai,Slimak,MistButterfly,kioniel,Xinil,ihasabucket,xTheFallenx,Lithuelle,...,Rinato,Otaku_Lodai,Byakuya1619,tsuparkle,ryugan,purgatorio,Kagekori-chan,Sicka,winry-chan,badking95
